<a href="https://colab.research.google.com/github/rishavganguly007/Identifying-Significant-Associations-in-Gene-Expression-Data-for-Both-Normal-and-Carcinogenic-State/blob/main/Final_Yr_Prjkt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files 
uploaded = files.upload()

Saving gene_expression_data_for_Lung.xlsx to gene_expression_data_for_Lung.xlsx


##**Pre-Processing**##

In [ ]:
import pandas as pd
import numpy as np  
import seaborn as sns 
import matplotlib.pyplot as plt
!pip install apyori # Install Package
from apyori import apriori
#/content/gene_expression_data_for_Lung.xlsx


  Created wheel for apyori: filename=apyori-1.1.2-cp37-none-any.whl size=5975 sha256=94f412b670a00fdbfaa0f0583da29eeeae0ceae7e52621d5a1bd611da7e6d0f1
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori


In [ ]:
df = pd.read_excel('gene_expression_data_for_Lung.xlsx')
df


,Unnamed: 0,Unnamed: 1,Sample ID,AD10,AD2,AD3,AD5,AD6,AD7,AD8,L01,L02,L04,L05,L06,L08,L09,L100,L101,L102,L103,L104,L105,L106,L107,L108,L11,L111,L12,L13,L17,L18,L19,L20,L22,L23,L24,L25,L26,L27,...,L53,L54,L56,L57,L59,L61,L62,L64,L65,L76,L78,L79,L80,L81,L82,L83,L84,L85,L86,L87,L88,L89,L90,L91,L92,L94,L95,L96,L97,L99,LN64,LN66,LN67,LN69,LN70,LN71,LN72B,LN73,LN74,LN75
0,NaN,NaN,Cluster ID,49.0,64.0,63.0,60.0,30.0,56.0,18.0,66.0,47.0,68.0,65.0,67.0,46.0,43.0,21.0,22.0,17.0,12.0,11.0,61.0,58.0,31.0,10.0,29.0,8.0,2.0,50.0,86.0,59.0,34.0,20.0,13.0,35.0,75.0,74.0,82.0,78.0,...,38.0,6.0,85.0,36.0,71.0,1.0,33.0,39.0,62.0,72.0,7.0,27.0,28.0,16.0,54.0,25.0,42.0,26.0,41.0,53.0,23.0,5.0,14.0,32.0,24.0,70.0,69.0,4.0,57.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GENE,PROBESET,IN 4966 genes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GABRA3,A28102_at,YES,170.0,59.7,80.0,92.4,104.0,88.0,69.7,230.0,105.0,53.7,96.9,104.0,91.0,83.0,845.4,265.9,343.7,262.0,163.0,128.4,121.4,729.4,284.3,119.0,191.6,32.9,105.5,93.0,85.8,130.0,58.0,284.2,141.7,195.9,148.3,116.2,78.1,...,108.0,455.8,71.0,133.2,76.0,141.7,77.4,149.0,78.0,149.1,166.0,174.9,246.5,66.2,57.0,219.6,197.1,90.0,71.1,98.0,299.8,339.1,277.5,73.0,133.5,78.1,68.0,35.5,79.8,57.0,108.0,108.0,170.8,88.0,97.0,89.1,77.2,198.4,180.6,106.2
3,OMD,AB000114_at,YES,69.4,18.1,26.0,96.9,72.8,138.6,11.1,176.0,78.1,36.7,28.4,70.0,91.0,78.0,126.6,17.8,44.1,99.0,76.4,54.1,80.0,137.0,236.3,42.6,86.0,32.0,63.7,41.4,42.6,90.2,30.8,84.4,148.7,65.0,56.2,67.6,52.8,...,228.3,217.0,26.0,125.0,27.2,52.6,216.2,123.0,11.8,81.6,51.1,127.8,112.0,24.1,36.2,61.0,43.5,19.1,60.4,73.8,401.6,204.0,309.4,96.9,219.2,3.0,27.2,221.5,37.8,7.9,93.0,103.5,100.5,72.8,118.0,118.0,54.9,110.7,243.9,164.0
4,GS3686,AB000115_at,YES,250.7,146.8,150.0,177.8,228.7,115.5,177.8,511.3,233.9,393.6,339.5,331.0,165.5,922.6,651.0,243.0,119.7,866.5,193.0,149.0,240.1,214.9,113.0,918.4,342.0,233.5,335.3,167.4,122.3,222.1,121.7,272.7,519.3,348.9,137.5,188.9,158.0,...,2485.2,544.0,220.0,272.9,104.0,128.0,612.5,675.7,148.0,150.4,690.6,192.1,183.3,71.6,117.2,792.9,467.0,114.4,214.9,606.4,513.3,515.5,1026.4,217.6,471.4,132.3,110.7,492.9,180.6,64.3,152.8,161.4,292.1,109.8,215.8,325.9,173.1,282.2,263.2,264.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7126,ATP7A,Z94753_s_at,YES,91.8,113.0,143.1,398.1,205.2,117.2,77.4,206.0,226.6,103.0,138.2,135.1,112.5,152.4,490.2,177.0,265.0,191.8,320.3,217.1,184.4,58.8,433.1,98.0,224.0,112.2,103.8,114.9,184.7,232.5,118.0,17.4,362.2,326.0,161.7,177.4,138.4,...,219.6,82.5,243.0,285.6,211.4,70.4,411.0,52.7,150.0,175.1,303.5,365.8,219.2,225.6,158.8,346.9,116.3,96.0,197.6,287.1,261.2,86.4,266.9,127.8,95.5,187.9,167.0,23.5,155.2,188.0,310.8,208.6,148.3,142.8,304.2,126.4,107.2,92.0,164.9,155.8
7127,NaN,Z95624_at,YES,45.0,101.3,58.3,81.5,63.9,39.0,81.0,35.5,29.0,18.9,39.0,22.0,63.9,73.0,113.3,98.0,178.0,185.7,92.4,67.6,73.6,185.3,233.0,72.0,99.8,79.0,50.0,49.0,69.4,78.2,202.4,143.1,53.4,65.0,25.2,41.0,41.4,...,194.0,254.5,73.8,116.0,33.4,95.0,65.7,88.0,68.0,89.2,108.0,88.0,65.0,243.0,66.0,91.3,68.5,101.6,82.0,74.6,92.3,72.6,179.1,43.8,116.1,68.5,56.3,56.2,73.6,60.3,44.0,45.9,72.8,54.9,59.7,70.2,69.4,80.1,93.5,98.0
7128,SLC6A14,Z96810_at,YES,49.0,47.3,155.0,24.8,160.1,9.0,-14.2,-21.0,69.4,57.0,9.0,455.8,20.2,311.7,-22.5,-4.0,61.7,-7.5,-2.0,19.3,187.2,67.9,30.0,-3.0,85.0,45.0,37.6,1729.3,13.7,29.4,72.0,507.2,291.2,53.9,-26.0,99.0,5.4,...,39.6,-10.4,8.0,162.6,179.3,59.0,59.0,-34.8,3.0,29.6,58.

In [ ]:
new_df = df.drop(index=[1,0])
#new_df

new_df.reset_index(drop=True)
df1 = new_df 
#df1

df1.drop(df1.iloc[:, 1:3], inplace = True, axis = 1)
#df1

In [ ]:
gp = df1.iloc[:,0].to_numpy()
#gp

gn =["Gene"+str(x) if gp[x] is np.nan else gp[x] for x in range(len(gp))]
gn = np.asarray(gn)

In [ ]:
df_safe = df1.iloc[:,-10:]
df_safe

,LN64,LN66,LN67,LN69,LN70,LN71,LN72B,LN73,LN74,LN75
2,108.0,108.0,170.8,88.0,97.0,89.1,77.2,198.4,180.6,106.2
3,93.0,103.5,100.5,72.8,118.0,118.0,54.9,110.7,243.9,164.0
4,152.8,161.4,292.1,109.8,215.8,325.9,173.1,282.2,263.2,264.0
5,387.3,347.2,702.8,643.8,368.9,429.4,622.7,507.9,896.6,529.0
6,32.0,3.9,-1.7,19.3,39.0,-4.0,-1.8,-16.4,12.5,3.0
...,...,...,...,...,...,...,...,...,...,...
7126,310.8,208.6,148.3,142.8,304.2,126.4,107.2,92.0,164.9,155.8
7127,44.0,45.9,72.8,54.9,59.7,70.2,69.4,80.1,93.5,98.0
7128,15.1,14.0,-23.4,-4.6,0.4,63.7,42.0,56.4,67.6,-1.7
7129,572.8,519.5,484.3,433.7,502.7,477.9,454.3,415.7,426.5,447.0


In [ ]:
df_unsafe = df1.iloc[:,1:-10]
df_unsafe

,AD10,AD2,AD3,AD5,AD6,AD7,AD8,L01,L02,L04,L05,L06,L08,L09,L100,L101,L102,L103,L104,L105,L106,L107,L108,L11,L111,L12,L13,L17,L18,L19,L20,L22,L23,L24,L25,L26,L27,L30,L31,L33,...,L41,L42,L43,L45,L46,L47,L48,L49,L50,L52,L53,L54,L56,L57,L59,L61,L62,L64,L65,L76,L78,L79,L80,L81,L82,L83,L84,L85,L86,L87,L88,L89,L90,L91,L92,L94,L95,L96,L97,L99
2,170.0,59.7,80.0,92.4,104.0,88.0,69.7,230.0,105.0,53.7,96.9,104.0,91.0,83.0,845.4,265.9,343.7,262.0,163.0,128.4,121.4,729.4,284.3,119.0,191.6,32.9,105.5,93.0,85.8,130.0,58.0,284.2,141.7,195.9,148.3,116.2,78.1,294.6,149.4,135.8,...,121.7,129.0,72.9,106.5,102.5,95.3,90.2,57.0,89.1,72.8,108.0,455.8,71.0,133.2,76.0,141.7,77.4,149.0,78.0,149.1,166.0,174.9,246.5,66.2,57.0,219.6,197.1,90.0,71.1,98.0,299.8,339.1,277.5,73.0,133.5,78.1,68.0,35.5,79.8,57.0
3,69.4,18.1,26.0,96.9,72.8,138.6,11.1,176.0,78.1,36.7,28.4,70.0,91.0,78.0,126.6,17.8,44.1,99.0,76.4,54.1,80.0,137.0,236.3,42.6,86.0,32.0,63.7,41.4,42.6,90.2,30.8,84.4,148.7,65.0,56.2,67.6,52.8,74.8,241.2,89.0,...,99.8,121.0,27.2,99.5,33.0,25.0,13.7,42.0,70.2,30.5,228.3,217.0,26.0,125.0,27.2,52.6,216.2,123.0,11.8,81.6,51.1,127.8,112.0,24.1,36.2,61.0,43.5,19.1,60.4,73.8,401.6,204.0,309.4,96.9,219.2,3.0,27.2,221.5,37.8,7.9
4,250.7,146.8,150.0,177.8,228.7,115.5,177.8,511.3,233.9,393.6,339.5,331.0,165.5,922.6,651.0,243.0,119.7,866.5,193.0,149.0,240.1,214.9,113.0,918.4,342.0,233.5,335.3,167.4,122.3,222.1,121.7,272.7,519.3,348.9,137.5,188.9,158.0,834.5,443.8,142.8,...,255.0,196.0,176.4,339.0,253.1,180.2,234.8,321.9,491.0,180.4,2485.2,544.0,220.0,272.9,104.0,128.0,612.5,675.7,148.0,150.4,690.6,192.1,183.3,71.6,117.2,792.9,467.0,114.4,214.9,606.4,513.3,515.5,1026.4,217.6,471.4,132.3,110.7,492.9,180.6,64.3
5,957.1,186.8,340.2,515.8,540.8,616.6,380.5,523.9,602.7,160.5,880.5,1766.0,442.5,340.4,769.4,458.0,155.7,473.8,330.3,312.3,168.0,1166.4,405.3,703.4,434.5,274.9,1768.8,292.3,286.2,1054.4,339.4,362.6,503.1,125.6,261.9,177.4,56.2,537.8,512.7,467.7,...,785.6,449.5,447.8,464.8,783.3,614.4,602.0,479.2,720.8,469.7,1204.7,1073.9,456.8,516.5,413.5,1232.7,1061.5,600.2,467.1,134.7,1250.0,522.3,426.3,84.2,574.8,253.5,448.8,274.2,347.1,731.7,532.1,3042.7,393.5,769.8,506.9,521.8,425.1,433.0,442.8,373.9
6,25.4,-7.7,-16.3,18.0,26.0,9.0,21.0,32.0,24.3,27.0,19.0,26.0,29.0,18.5,-8.0,19.6,163.8,43.3,19.8,-6.5,-35.1,33.2,25.5,-5.3,19.8,26.0,29.6,9.0,21.8,22.2,17.5,12.0,75.5,-15.5,9.0,40.0,4.2,12.2,52.0,26.0,...,73.0,46.0,19.0,14.6,19.6,-0.6,6.0,-8.2,3.7,9.0,-1.1,56.6,12.5,-30.7,34.8,32.0,69.9,12.5,6.0,20.0,-1.0,-16.3,9.8,220.3,-7.0,15.3,14.0,5.7,14.0,16.8,83.2,84.7,27.8,13.2,97.2,-3.0,18.5,34.0,-12.8,26.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7126,91.8,113.0,143.1,398.1,205.2,117.2,77.4,206.0,226.6,103.0,138.2,135.1,112.5,152.4,490.2,177.0,265.0,191.8,320.3,217.1,184.4,58.8,433.1,98.0,224.0,112.2,103.8,114.9,184.7,232.5,118.0,17.4,362.2,326.0,161.7,177.4,138.4,55.0,270.1,151.0,...,81.0,355.5,119.7,84.4,98.9,83.0,239.4,149.2,166.0,199.9,219.6,82.5,243.0,285.6,211.4,70.4,411.0,52.7,150.0,175.1,303.5,365.8,219.2,225.6,158.8,346.9,116.3,96.0,197.6,287.1,261.2,86.4,266.9,127.8,95.5,187.9,167.0,23.5,155.2,188.0
7127,45.0,101.3,58.3,81.5,63.9,39.0,81.0,35.5,29.0,18.9,39.0,22.0,63.9,73.0,113.3,98.0,178.0,185.7,92.4,67.6,73.6,185.3,233.0,72.0,99.8,79.0,50.0,49.0,69.4,78.2,202.4,143.1,53.4,65.0,25.2,41.0,41.4,162.9,102.0,57.0,...,98.0,198.0,27.2,49.7,96.0,58.1,83.5,55.6,109.7,110.3,194.0,254.5,73.8,116.0,33.4,95.0,65.7,88.0,68.0,89.2,108.0,88.0,65.0,243.0,66.0,91.3,68.5,101.6,82.0,74.6,92.3,72.6,179.1,43.8,116.1,68.5,56.3,56.2,73.6,60.3
7128,49.0,47.3,155.0,24.8,160.1,9.0,-14.2,-21.0,69.4,57.0,9.0,455.8,20.2,311.7,-22.5,-4.0,61.7,-7.5,-2.0,19.3,187.2,67.9,30.0,-3.0,85.0,45.0,37.6,1729.3,13.7,29.4,72.0,507.2,2

##**Working on Normal Dataset**##

In [ ]:
df_safe_trans = df_safe.transpose()
df_safe_trans

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,...,7091,7092,7093,7094,7095,7096,7097,7098,7099,7100,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129,7130
LN64,108.0,93.0,152.8,387.3,32.0,237.4,-57.3,119.0,214.0,642.8,152.8,108.8,-135.0,24.6,587.8,653.7,75.5,156.2,88.8,1.5,75.5,338.0,740.0,75.5,199.0,614.1,285.2,525.5,85.0,26.9,419.2,599.2,576.3,10013.8,360.3,413.2,329.8,572.0,143.0,519.0,...,9.0,-65.0,11.6,35.5,77.2,-42.8,-22.7,-41.8,27.7,-6.2,-3.0,12.5,-46.6,39.0,437.2,0.8,-6.8,-8.7,53.4,-81.3,20.2,40.0,-146.4,0.0,-26.8,12.5,115.5,72.9,422.2,-70.8,354.9,5736.4,134.0,304.6,279.7,310.8,44.0,15.1,572.8,180.0
LN66,108.0,103.5,161.4,347.2,3.9,248.7,40.0,35.5,186.0,424.1,110.5,181.0,-215.8,55.1,664.6,381.6,26.9,270.4,133.5,32.0,179.0,323.7,496.1,72.9,205.2,533.3,220.4,604.0,155.8,17.7,219.6,517.3,636.8,9703.3,317.0,450.4,268.8,631.6,112.2,485.8,...,-14.6,-26.4,-56.0,56.4,123.0,-11.8,17.7,-32.8,25.3,-3.8,23.9,51.1,-2.2,12.0,464.2,-6.1,37.8,2.2,14.0,-29.6,-14.6,37.8,-138.1,-17.3,-6.9,8.1,101.6,80.1,466.8,160.3,336.3,3056.2,102.5,347.2,288.0,208.6,45.9,14.0,519.5,154.8
LN67,170.8,100.5,292.1,702.8,-1.7,377.2,-34.4,132.2,273.0,1070.3,263.0,244.0,-144.5,-10.0,767.4,583.0,-1.0,198.0,98.0,65.9,120.8,623.8,946.1,84.0,272.1,604.2,247.0,532.6,365.4,71.1,476.1,843.3,1006.8,9018.6,479.7,496.1,328.5,821.9,113.5,608.4,...,-10.0,-41.5,-17.6,51.0,48.0,-25.7,5.0,12.0,-85.1,63.9,8.3,18.1,-1.7,20.3,431.0,-20.2,4.3,-12.5,22.7,-65.1,23.5,13.0,-153.6,18.9,-61.5,-0.3,101.8,17.4,405.1,-70.2,370.1,5678.9,115.3,402.2,289.0,148.3,72.8,-23.4,484.3,138.6
LN69,88.0,72.8,109.8,643.8,19.3,363.0,13.7,151.9,193.6,599.3,118.9,196.2,-137.6,-0.4,570.2,852.3,67.0,224.9,117.1,59.3,87.1,425.0,820.0,118.0,218.3,406.1,200.7,466.3,97.0,3.0,433.0,795.7,630.0,8647.2,419.7,424.3,330.7,555.7,108.0,429.7,...,0.4,-58.8,1.3,99.1,114.4,8.2,-37.0,-28.5,-7.7,3.0,16.8,26.0,-10.0,50.0,425.0,11.3,-0.4,-2.1,-1.3,-71.3,-10.0,46.0,-144.0,79.9,-40.1,20.2,117.1,44.1,349.5,5.2,321.6,6059.2,82.6,283.9,313.1,142.8,54.9,-4.6,433.7,181.0
LN70,97.0,118.0,215.8,368.9,39.0,390.9,2.1,110.5,237.0,552.4,158.0,262.2,-118.6,106.2,610.7,476.3,43.0,238.0,72.0,36.7,117.2,291.4,779.9,55.3,201.6,394.9,236.0,567.0,44.0,43.0,482.3,965.7,339.3,8391.9,439.0,452.2,323.8,564.6,63.7,528.8,...,-14.0,-141.8,-12.0,52.8,112.2,10.6,11.3,8.0,50.0,-8.6,1.3,49.0,-2.1,-18.0,451.3,-3.0,48.0,7.0,12.9,-5.8,21.0,-13.0,-107.0,17.7,-52.2,3.0,66.8,19.3,339.3,799.5,414.3,8204.3,181.5,256.8,259.5,304.2,59.7,0.4,502.7,160.3
LN71,89.1,118.0,325.9,429.4,-4.0,318.8,57.0,112.6,233.4,578.8,135.4,254.0,-171.0,22.7,531.6,148.7,46.0,197.7,98.8,-14.6,121.6,442.8,477.0,59.7,198.3,475.2,158.0,560.4,91.3,34.3,430.3,743.9,926.3,10103.0,432.8,480.6,274.0,640.4,156.1,472.5,...,-8.0,-69.1,27.0,56.3,118.0,-8.0,-5.0,-11.8,14.0,16.0,37.8,22.7,-31.0,42.8,468.8,-37.1,-2.1,-18.2,16.0,-51.6,-5.0,6.4,-165.8,33.2,15.0,-15.5,32.0,52.7,366.5,274.0,325.0,3828.3,112.6,226.5,364.8,126.4,70.2,63.7,477.9,158.0
LN72B,77.2,54.9,173.1,622.7,-1.8,502.8,45.0,69.4,153.6,515.9,112.2,135.4,3.0,52.7,448.8,1475.3,54.9,217.8,39.0,53.4,69.4,597.2,542.7,52.0,229.7,421.1,379.3,439.4,123.6,60.0,761.5,981.6,896.7,7962.1,452.5,507.2,429.3,713.5,64.0,519.5,...,22.0,-87.6,-36.0,64.0,99.7,-22.4,3.0,-8.8,88.9,10.2,53.4,1.8,27.2,108.0,274.2,-19.8,3.0,-41.3,3.0,-55.4,-12.4,29.6,-123.6,24.0,-14.9,-0.6,80.8,-1.8,415.4,643.9,339.3,3085.8,158.9,209.7,426.8,107.2,69.4,42.0,454.3,175.9
LN73,198.4,110.7,282.2,507.9,-16.4,323.6,23.5,180.4,219.4,634.0,150.5,261.6,-113.0,10.0,473.7,649.9,65.0,141.8,85.8,56.4,83.5,405.4,935.4,78.1,206.0,575.8,196.7,490.3,99.7,42.4,497.7,774.6,1092.3,8103.2,320.8,430.5,299.0,539.1,26.9,393.4,...,-19.2,-74.7,-10.9,48.0,78.1,-38.4,-27.9,41.6,29.4,30.3,-73.6,93.0,14.0,82.4,457.8,-21.0,-3.0,14.0,43.3,-30.2,9.0,33.8,-139.2,40.7,-10.9,-44.4,102.2,21.8,269.9,564.2,285.5,6307.4,200.9,350.6,283.1,92.0,80.1,56.4,415.

In [ ]:
#NORMALISATION
from sklearn.preprocessing import StandardScaler
# create a scaler object
std_scaler = StandardScaler()
std_scaler
# fit and transform the data
df3 = pd.DataFrame(std_scaler.fit_transform(df_safe_trans), columns=df_safe_trans.columns)

df3

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,...,7091,7092,7093,7094,7095,7096,7097,7098,7099,7100,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129,7130
0,-0.345896,-0.496754,-1.047580,-0.943481,1.437899,-1.446714,-2.019058,-0.064931,-0.224723,-0.201698,0.087911,-1.957274,-0.242048,-0.091929,0.203486,0.011489,1.344117,-1.035816,-0.101773,-1.501949,-1.089956,-1.004438,-0.009339,-0.086504,-0.961117,1.443596,0.626667,0.081031,-0.570834,-0.621092,-0.570219,-1.187964,-0.872398,0.970042,-0.765817,-1.560878,0.177056,-0.646155,1.280256,0.338964,...,1.187792,0.309747,0.818829,-1.278152,-0.458859,-1.451923,-0.982873,-1.414777,0.362800,-0.936786,-0.465843,-0.863059,-1.639904,-0.152849,0.104587,0.710730,-0.856686,-0.153818,1.286819,-1.547057,0.270870,0.318341,-0.611253,-1.030584,-0.270990,0.917751,0.852323,1.413774,0.680285,-0.998020,0.204349,-0.028395,-0.304001,-0.058420,-0.756072,1.862244,-1.428967,-0.257396,2.159497,1.352550
1,-0.345896,-0.287531,-0.921099,-1.185600,-0.287334,-1.293213,0.511592,-1.967291,-0.947971,-1.321277,-0.928112,-0.523184,-1.714749,0.812534,1.026864,-0.766157,-0.773357,1.838794,1.740036,-0.283613,2.028940,-1.119327,-1.469417,-0.225338,-0.703712,0.397193,-0.428910,1.777284,0.203697,-1.054631,-1.934914,-1.578249,-0.643059,0.614529,-1.597726,-0.425916,-0.875618,0.052771,0.445542,-0.248078,...,-0.803109,1.585759,-2.068642,-0.238316,0.957296,-0.174330,1.045119,-1.087955,0.307865,-0.837655,0.223441,0.636900,0.354938,-0.782912,0.569803,0.226141,1.103715,0.594670,-0.571710,0.904900,-1.091310,0.227973,-0.286867,-1.659256,0.591842,0.659230,0.329264,1.710370,1.490019,-0.309165,-0.351337,-1.454350,-0.881219,0.656716,-0.598081,0.449316,-1.319710,-0.293464,1.001071,-0.023480
2,1.169962,-0.347309,1.001105,0.961473,-0.631153,0.452353,-1.423458,0.235801,1.299263,1.986779,2.734855,0.728168,-0.415200,-1.117976,2.128990,-0.190567,-1.988944,0.016362,0.277301,1.070538,0.275126,1.291741,1.224455,0.367376,2.073770,1.315386,0.004398,0.234450,2.496661,1.461781,-0.181185,-0.024732,0.759510,-0.169433,1.528177,0.968379,0.154622,2.284412,0.480774,1.919734,...,-0.415052,1.086594,-0.428422,-0.506982,-1.361735,-0.747186,0.407606,0.538894,-2.219143,1.958659,-0.176293,-0.645449,0.377403,-0.589226,-0.002240,-0.764105,-0.368784,-0.414758,-0.161324,-0.778745,0.400043,-0.790717,-0.892648,-0.343770,-1.775527,0.165689,0.336790,-0.872486,0.369827,-0.996232,0.658458,-0.058987,-0.646667,1.580013,-0.579046,-0.384338,0.227140,-1.519781,0.236032,-0.908071
3,-0.828653,-0.899257,-1.679980,0.605238,0.658167,0.259458,-0.172438,0.684622,-0.751661,-0.424385,-0.726348,-0.221271,-0.289437,-0.833292,0.014795,0.579076,0.973777,0.693481,1.064294,0.806898,-0.740399,-0.305461,0.469572,2.182898,-0.159840,-1.250115,-0.749818,-1.198181,-0.439558,-1.747351,-0.475866,-0.251565,-0.668836,-0.594675,0.375416,-1.222220,0.192588,-0.837305,0.331718,-1.240038,...,0.462294,0.514702,0.378874,1.886133,0.691380,0.649924,-1.700702,-0.931806,-0.447491,-0.556785,0.041511,-0.338462,0.004493,0.103844,-0.105621,1.448148,-0.575373,0.299395,-1.293423,-1.072790,-0.911252,0.564798,-0.517454,1.872931,-0.847657,1.370164,0.912531,0.227390,-0.639617,-0.771482,-0.790508,0.143346,-1.245875,-0.405915,-0.120302,-0.460377,-0.802177,-0.903344,-0.863712,1.407155
4,-0.611412,0.001395,-0.121039,-1.054578,1.867673,0.638456,-0.474139,-0.258584,0.369373,-0.664478,0.212812,1.089670,0.056867,2.327881,0.448998,-0.495510,-0.071890,1.023230,-0.793997,-0.095869,0.166642,-1.378832,0.229518,-1.165138,-0.853173,-1.395161,-0.174789,0.977777,-1.019362,0.137602,-0.138794,0.558551,-1.770801,-0.886986,0.746220,-0.370998,0.073515,-0.732935,-0.868861,0.512247,...,-0.752493,-2.229055,-0.189223,-0.417427,0.623356,0.748834,0.723853,0.393639,0.873237,-1.035917,-0.355660,0.555296,0.359431,-1.482982,0.347533,0.443855,1.552057,0.924279,-0.623598,2.033654,0.302185,-1.858699,0.

In [ ]:
for i in range(0,len(df_safe_trans)) :
  for j in range(0,len(df_safe_trans.columns)):
    if -1.0< df3.iloc[i,j] > 0.0:
      df3.iloc[i, j] = gn[j]
    else:
      df3.iloc[i, j] = 0
print(df3)

     2    3       4       5    6     ...   7126      7127     7128   7129     7130
0       0    0       0       0  GML  ...  ATP7A         0        0  UREB1  RAB9P40
1       0    0       0       0    0  ...  ATP7A         0        0  UREB1        0
2  GABRA3    0  GS3686  SEMA3C    0  ...      0  Gene7125        0  UREB1        0
3       0    0       0  SEMA3C  GML  ...      0         0        0      0  RAB9P40
4       0  OMD       0       0  GML  ...  ATP7A         0        0  UREB1  RAB9P40
5       0  OMD  GS3686       0    0  ...      0  Gene7125  SLC6A14  UREB1  RAB9P40
6       0    0       0  SEMA3C    0  ...      0  Gene7125  SLC6A14      0  RAB9P40
7  GABRA3    0  GS3686       0    0  ...      0  Gene7125  SLC6A14      0        0
8  GABRA3  OMD  GS3686  SEMA3C  GML  ...      0  Gene7125  SLC6A14      0        0
9       0  OMD  GS3686       0    0  ...      0  Gene7125        0      0        0

[10 rows x 7129 columns]


In [ ]:
df3_safe = df3.replace(0, np.nan)
df3_safe

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,...,7091,7092,7093,7094,7095,7096,7097,7098,7099,7100,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129,7130
0,NaN,NaN,NaN,NaN,GML,NaN,NaN,NaN,NaN,NaN,SH3BP2,NaN,NaN,NaN,RNF4,PLAB,ARNTL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KIAA0320,CLOCK,MADD,NaN,NaN,NaN,NaN,NaN,RPLP2,NaN,NaN,PSMD12,NaN,CDC7L1,TLK2,...,GFPT1,FLJ11618,Gene7091,NaN,NaN,NaN,NaN,NaN,H2BFH,NaN,NaN,NaN,NaN,NaN,H2BFD,H3FK,NaN,NaN,PPP1R11,NaN,CKS1,DNAH7,NaN,NaN,NaN,DNAH9,CYB5-M,KL,BRD2,NaN,HBZ,NaN,NaN,NaN,NaN,ATP7A,NaN,NaN,UREB1,RAB9P40
1,NaN,NaN,NaN,NaN,NaN,NaN,OGG1,NaN,NaN,NaN,NaN,NaN,NaN,RES4-25,RNF4,NaN,NaN,CDH23,PCDHGB4,NaN,H4FM,NaN,NaN,NaN,NaN,KIAA0320,NaN,MADD,KIAA0367,NaN,NaN,NaN,NaN,RPLP2,NaN,NaN,NaN,PSMD9,CDC7L1,NaN,...,NaN,FLJ11618,NaN,NaN,ACADS,NaN,Gene7095,NaN,H2BFH,NaN,TPM1,H4FJ,H4FL,NaN,H2BFD,H3FK,H2AFM,H2AFC,NaN,DNAH9,NaN,DNAH7,NaN,NaN,DNAH9,DNAH9,CYB5-M,KL,BRD2,NaN,NaN,NaN,NaN,SSTR3,NaN,ATP7A,NaN,NaN,UREB1,NaN
2,GABRA3,NaN,GS3686,SEMA3C,NaN,MKNK1,NaN,VRK1,VRK2,RES4-22,SH3BP2,Gene11,NaN,NaN,RNF4,NaN,NaN,CDH23,PCDHGB4,PCDHGA12,H4FM,GMFB,AQP3,KIAA0316,KIAA0317,KIAA0320,CLOCK,MADD,KIAA0367,KIAA0368,NaN,NaN,SCYA21,NaN,STXBP2,PSMD11,PSMD12,PSMD9,CDC7L1,TLK2,...,NaN,FLJ11618,NaN,NaN,NaN,NaN,Gene7095,H1FT,NaN,Gene7098,NaN,NaN,H4FL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CKS1,NaN,NaN,NaN,NaN,DNAH9,CYB5-M,NaN,BRD2,NaN,HBZ,NaN,NaN,SSTR3,NaN,NaN,Gene7125,NaN,UREB1,NaN
3,NaN,NaN,NaN,SEMA3C,GML,MKNK1,NaN,VRK1,NaN,NaN,NaN,NaN,NaN,NaN,RNF4,PLAB,ARNTL,CDH23,PCDHGB4,PCDHGA12,NaN,NaN,AQP3,KIAA0316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STXBP2,NaN,PSMD12,NaN,CDC7L1,NaN,...,GFPT1,FLJ11618,Gene7091,PTPRR,ACADS,H2BFQ,NaN,NaN,NaN,NaN,TPM1,NaN,H4FL,SERPINI1,NaN,H3FK,NaN,H2AFC,NaN,NaN,NaN,DNAH7,NaN,DNAH9,NaN,DNAH9,CYB5-M,KL,NaN,NaN,NaN,HBA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RAB9P40
4,NaN,OMD,NaN,NaN,GML,MKNK1,NaN,NaN,VRK2,NaN,SH3BP2,Gene11,Gene12,RES4-25,RNF4,NaN,NaN,CDH23,NaN,NaN,H4FM,NaN,AQP3,NaN,NaN,NaN,NaN,MADD,NaN,KIAA0368,NaN,CTNND1,NaN,NaN,STXBP2,NaN,PSMD12,NaN,NaN,TLK2,...,NaN,NaN,NaN,NaN,ACADS,H2BFQ,Gene7095,H1FT,H2BFH,NaN,NaN,H4FJ,H4FL,NaN,H2BFD,H3FK,H2AFM,H2AFC,NaN,DNAH9,CKS1,NaN,DNAH9,NaN,NaN,DNAH9,NaN,NaN,NaN,GSTT1,HBZ,HBA2,Gene7121,NaN,NaN,ATP7A,NaN,NaN,UREB1,RAB9P40
5,NaN,OMD,GS3686,NaN,NaN,NaN,OGG1,NaN,VRK2,NaN,NaN,Gene11,NaN,NaN,NaN,NaN,ARNTL,CDH23,PCDHGB4,NaN,H4FM,NaN,NaN,NaN,NaN,NaN,NaN,MADD,NaN,NaN,NaN,NaN,SCYA21,RPLP2,STXBP2,PSMD11,NaN,PSMD9,CDC7L1,NaN,...,NaN,FLJ11618,Gene7091,NaN,ACADS,NaN,NaN,NaN,H2BFH,NaN,TPM1,NaN,NaN,NaN,H2BFD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DNAH9,DNAH9,NaN,NaN,KL,NaN,GSTT1,NaN,NaN,NaN,NaN,E46L,NaN,Gene7125,SLC6A14,UREB1,RAB9P40
6,NaN,NaN,NaN,SEMA3C,NaN,MKNK1,OGG1,NaN,NaN,NaN,NaN,NaN,Gene12,RES4-25,NaN,PLAB,ARNTL,CDH23,NaN,PCDHGA12,NaN,GMFB,NaN,NaN,KIAA0317,NaN,CLOCK,NaN,NaN,KIAA0368,ARHGEF12,CTNND1,SCYA21,NaN,STXBP2,PSMD11,PSMD12,PSMD9,NaN,TLK2,...,GFPT1,NaN,NaN,PTPRR,ACADS,NaN,Gene7095,NaN,H2BFH,NaN,TPM1,NaN,H4FL,SERPINI1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DNAH9,NaN,DNAH9,DNAH9,NaN,NaN,BRD2,GSTT1,NaN,NaN,Gene7121,NaN,E46L,NaN,Gene7125,SLC6A14,NaN,RAB9P40
7,GABRA3,NaN,GS3686,NaN,NaN,NaN,OGG1,VRK1,NaN,NaN,SH3BP2,Gene11,Gene12,NaN,NaN,PLAB,ARNTL,NaN,NaN,PCDHGA12,NaN,NaN,AQP3,KIAA0316,NaN,KIAA0320,NaN,NaN,NaN,KIAA0368,NaN,NaN,SCYA21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H1FT,H2BFH,Gene7098,NaN,H4FJ,H4FL,SERPINI1,H2BFD,NaN,NaN,H2AFC,PPP1R11,DNAH9,NaN,DNAH7,NaN,DNAH9,DNAH9,NaN,CYB5-M,NaN,NaN,GSTT1,NaN,HBA2,Gene7121,SSTR3,NaN,NaN,Gene7125,SLC6A14,NaN,NaN
8,GABRA3,OMD,GS3686,SEMA3C,GML,NaN,OGG1,VRK1,VRK2,RES4-22,NaN,Gene11,Gene12,NaN,NaN,NaN,NaN,NaN,PCDHGB4,PCDHGA12,NaN,GMFB,AQP3,KIAA0316,KIAA0317,NaN,NaN,MADD,KIAA0367,NaN,ARHGEF12,CTNND1,SCYA21,NaN,NaN,PSMD11,NaN,NaN,NaN,TLK2,...,NaN,NaN,Gene7091,PTPRR,NaN,H2BFQ,NaN,H1FT,NaN,Gene7098,NaN,H4FJ,NaN,SERPINI1,H2BFD,H3FK,NaN,H2AFC,PPP1R11,NaN,CKS1,DNAH7,DNAH9,DNAH9,DNAH9,DNAH9,NaN,KL,BRD2,NaN,

In [ ]:
records = []
for i in range(0, len(df3_safe)):
    records.append([str(df3_safe.values[i,j]) for j in range(0, len(df3_safe.columns))])

In [ ]:
association_rules = apriori(records, min_support=0.005, min_confidence=0.3, min_lift=3, min_length=2,max_length = 2)
association_results = list(association_rules)

In [ ]:
print(len(association_results))

5983


In [ ]:
support_safe = []
conf_safe = []

for item in association_results:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))
    support_safe.append(float(item[1]))
    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    conf_safe.append(float(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

Streaming output truncated to the last 5000 lines.
Rule: MMP1 -> MPP1
Support: 0.1
Confidence: 1.0
Lift: 3.3333333333333335
Rule: MMP1 -> MRE11A
Support: 0.1
Confidence: 1.0
Lift: 3.3333333333333335
Rule: MTHFR -> MMP1
Support: 0.1
Confidence: 1.0
Lift: 3.3333333333333335
Rule: MMP1 -> NPAS1
Support: 0.1
Confidence: 1.0
Lift: 3.3333333333333335
Rule: MMP1 -> NR2F2
Support: 0.1
Confidence: 1.0
Lift: 3.3333333333333335
Rule: MMP1 -> NRCAM
Support: 0.1
Confidence: 1.0
Lift: 3.3333333333333335
Rule: NTF3 -> MMP1
Support: 0.1
Confidence: 1.0
Lift: 3.3333333333333335
Rule: NTF5 -> MMP1
Support: 0.1
Confidence: 1.0
Lift: 3.3333333333333335
Rule: MMP1 -> OAT
Support: 0.1
Confidence: 1.0
Lift: 3.3333333333333335
Rule: MMP1 -> ODF1
Support: 0.1
Confidence: 1.0
Lift: 3.3333333333333335
Rule: MMP1 -> P2RX4
Support: 0.1
Confidence: 1.0
Lift: 5.0
Rule: MMP1 -> P2RX7
Support: 0.1
Confidence: 1.0
Lift: 3.3333333333333335
Rule: MMP1 -> PCTK1
Support: 0.1
Confidence: 1.0
Lift: 3.3333333333333335
Rule: M

In [ ]:

with open("HEALTHY.txt", "w") as txt_file:
    for item in association_results:
      pair = item[0] 
      items = [x for x in pair]
      txt_file.write("Rule: " + items[0] + " -> " + items[1] + "\t"+"Support: " + str(item[1])+"\t"+"Confidence: " + str(item[2][0][2])+  "\n")


 

##**Working on Carcinogenic Dataset**##

In [ ]:
df_unsafe

,AD10,AD2,AD3,AD5,AD6,AD7,AD8,L01,L02,L04,L05,L06,L08,L09,L100,L101,L102,L103,L104,L105,L106,L107,L108,L11,L111,L12,L13,L17,L18,L19,L20,L22,L23,L24,L25,L26,L27,L30,L31,L33,...,L41,L42,L43,L45,L46,L47,L48,L49,L50,L52,L53,L54,L56,L57,L59,L61,L62,L64,L65,L76,L78,L79,L80,L81,L82,L83,L84,L85,L86,L87,L88,L89,L90,L91,L92,L94,L95,L96,L97,L99
2,170.0,59.7,80.0,92.4,104.0,88.0,69.7,230.0,105.0,53.7,96.9,104.0,91.0,83.0,845.4,265.9,343.7,262.0,163.0,128.4,121.4,729.4,284.3,119.0,191.6,32.9,105.5,93.0,85.8,130.0,58.0,284.2,141.7,195.9,148.3,116.2,78.1,294.6,149.4,135.8,...,121.7,129.0,72.9,106.5,102.5,95.3,90.2,57.0,89.1,72.8,108.0,455.8,71.0,133.2,76.0,141.7,77.4,149.0,78.0,149.1,166.0,174.9,246.5,66.2,57.0,219.6,197.1,90.0,71.1,98.0,299.8,339.1,277.5,73.0,133.5,78.1,68.0,35.5,79.8,57.0
3,69.4,18.1,26.0,96.9,72.8,138.6,11.1,176.0,78.1,36.7,28.4,70.0,91.0,78.0,126.6,17.8,44.1,99.0,76.4,54.1,80.0,137.0,236.3,42.6,86.0,32.0,63.7,41.4,42.6,90.2,30.8,84.4,148.7,65.0,56.2,67.6,52.8,74.8,241.2,89.0,...,99.8,121.0,27.2,99.5,33.0,25.0,13.7,42.0,70.2,30.5,228.3,217.0,26.0,125.0,27.2,52.6,216.2,123.0,11.8,81.6,51.1,127.8,112.0,24.1,36.2,61.0,43.5,19.1,60.4,73.8,401.6,204.0,309.4,96.9,219.2,3.0,27.2,221.5,37.8,7.9
4,250.7,146.8,150.0,177.8,228.7,115.5,177.8,511.3,233.9,393.6,339.5,331.0,165.5,922.6,651.0,243.0,119.7,866.5,193.0,149.0,240.1,214.9,113.0,918.4,342.0,233.5,335.3,167.4,122.3,222.1,121.7,272.7,519.3,348.9,137.5,188.9,158.0,834.5,443.8,142.8,...,255.0,196.0,176.4,339.0,253.1,180.2,234.8,321.9,491.0,180.4,2485.2,544.0,220.0,272.9,104.0,128.0,612.5,675.7,148.0,150.4,690.6,192.1,183.3,71.6,117.2,792.9,467.0,114.4,214.9,606.4,513.3,515.5,1026.4,217.6,471.4,132.3,110.7,492.9,180.6,64.3
5,957.1,186.8,340.2,515.8,540.8,616.6,380.5,523.9,602.7,160.5,880.5,1766.0,442.5,340.4,769.4,458.0,155.7,473.8,330.3,312.3,168.0,1166.4,405.3,703.4,434.5,274.9,1768.8,292.3,286.2,1054.4,339.4,362.6,503.1,125.6,261.9,177.4,56.2,537.8,512.7,467.7,...,785.6,449.5,447.8,464.8,783.3,614.4,602.0,479.2,720.8,469.7,1204.7,1073.9,456.8,516.5,413.5,1232.7,1061.5,600.2,467.1,134.7,1250.0,522.3,426.3,84.2,574.8,253.5,448.8,274.2,347.1,731.7,532.1,3042.7,393.5,769.8,506.9,521.8,425.1,433.0,442.8,373.9
6,25.4,-7.7,-16.3,18.0,26.0,9.0,21.0,32.0,24.3,27.0,19.0,26.0,29.0,18.5,-8.0,19.6,163.8,43.3,19.8,-6.5,-35.1,33.2,25.5,-5.3,19.8,26.0,29.6,9.0,21.8,22.2,17.5,12.0,75.5,-15.5,9.0,40.0,4.2,12.2,52.0,26.0,...,73.0,46.0,19.0,14.6,19.6,-0.6,6.0,-8.2,3.7,9.0,-1.1,56.6,12.5,-30.7,34.8,32.0,69.9,12.5,6.0,20.0,-1.0,-16.3,9.8,220.3,-7.0,15.3,14.0,5.7,14.0,16.8,83.2,84.7,27.8,13.2,97.2,-3.0,18.5,34.0,-12.8,26.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7126,91.8,113.0,143.1,398.1,205.2,117.2,77.4,206.0,226.6,103.0,138.2,135.1,112.5,152.4,490.2,177.0,265.0,191.8,320.3,217.1,184.4,58.8,433.1,98.0,224.0,112.2,103.8,114.9,184.7,232.5,118.0,17.4,362.2,326.0,161.7,177.4,138.4,55.0,270.1,151.0,...,81.0,355.5,119.7,84.4,98.9,83.0,239.4,149.2,166.0,199.9,219.6,82.5,243.0,285.6,211.4,70.4,411.0,52.7,150.0,175.1,303.5,365.8,219.2,225.6,158.8,346.9,116.3,96.0,197.6,287.1,261.2,86.4,266.9,127.8,95.5,187.9,167.0,23.5,155.2,188.0
7127,45.0,101.3,58.3,81.5,63.9,39.0,81.0,35.5,29.0,18.9,39.0,22.0,63.9,73.0,113.3,98.0,178.0,185.7,92.4,67.6,73.6,185.3,233.0,72.0,99.8,79.0,50.0,49.0,69.4,78.2,202.4,143.1,53.4,65.0,25.2,41.0,41.4,162.9,102.0,57.0,...,98.0,198.0,27.2,49.7,96.0,58.1,83.5,55.6,109.7,110.3,194.0,254.5,73.8,116.0,33.4,95.0,65.7,88.0,68.0,89.2,108.0,88.0,65.0,243.0,66.0,91.3,68.5,101.6,82.0,74.6,92.3,72.6,179.1,43.8,116.1,68.5,56.3,56.2,73.6,60.3
7128,49.0,47.3,155.0,24.8,160.1,9.0,-14.2,-21.0,69.4,57.0,9.0,455.8,20.2,311.7,-22.5,-4.0,61.7,-7.5,-2.0,19.3,187.2,67.9,30.0,-3.0,85.0,45.0,37.6,1729.3,13.7,29.4,72.0,507.2,2

In [ ]:
df_unsafe_trans = df_unsafe.transpose()
df_unsafe_trans

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,...,7091,7092,7093,7094,7095,7096,7097,7098,7099,7100,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129,7130
AD10,170.0,69.4,250.7,957.1,25.4,471.2,-52.0,42.8,200.9,846.4,134.7,147.0,-71.4,19.6,953.2,703.6,22.2,222.2,43.5,-7.0,95.5,526.9,777.5,62.3,149.0,565.7,400.6,554.6,68.5,22.2,281.6,1018.2,658.2,8787.4,287.1,452.6,210.9,580.5,139.3,460.7,...,55.6,-28.9,-10.8,99.0,-69.6,7.5,19.6,12.0,39.0,22.2,42.8,62.3,-0.6,86.2,485.7,58.8,1.8,-7.5,48.0,-58.3,3.0,27.0,-179.2,60.6,-92.4,4.5,11.0,16.0,510.4,-52.6,449.4,1385.2,50.0,335.6,256.4,91.8,45.0,49.0,498.5,276.0
AD2,59.7,18.1,146.8,186.8,-7.7,309.0,-99.0,57.9,151.5,722.8,55.3,131.2,-85.4,-44.0,552.1,2068.7,-22.0,178.3,69.0,-0.8,75.1,391.8,517.9,52.0,328.6,467.2,259.7,480.9,65.0,4.0,355.7,1579.4,419.8,7157.4,315.0,949.0,279.4,832.0,172.9,677.3,...,39.0,-89.3,16.0,102.4,-101.9,14.8,24.3,-6.9,961.0,79.0,48.5,210.0,-8.4,25.2,339.3,60.6,21.8,46.2,13.7,-45.0,-12.4,-2.2,-196.1,16.7,-134.0,20.3,38.0,68.0,526.1,-137.9,430.1,429.4,115.5,305.0,565.9,113.0,101.3,47.3,1102.8,134.7
AD3,80.0,26.0,150.0,340.2,-16.3,225.7,23.5,69.4,207.6,515.1,63.7,107.0,-78.3,49.2,609.4,447.0,30.8,99.0,53.4,28.4,68.5,288.9,1053.2,24.8,199.4,378.0,238.5,528.7,16.0,10.8,650.7,1254.4,319.3,9298.7,320.9,566.3,169.3,499.6,112.6,463.7,...,0.0,-45.5,-32.7,50.6,-24.3,-32.7,-7.7,-10.0,190.9,20.0,28.4,27.2,19.0,27.2,352.2,29.6,43.0,6.0,24.8,-62.8,-11.6,26.0,-151.2,34.8,-74.5,9.0,52.7,52.0,439.5,34.8,375.2,619.8,-17.9,292.7,445.3,143.1,58.3,155.0,691.8,243.8
AD5,92.4,96.9,177.8,515.8,18.0,296.6,48.5,60.4,151.5,819.1,56.3,118.9,-80.7,22.2,708.2,2771.2,75.5,111.6,67.6,4.2,57.0,326.1,4190.3,43.8,172.0,522.1,400.0,618.6,108.0,70.2,795.5,1293.3,172.0,10859.5,173.0,540.9,267.6,477.6,74.6,642.4,...,-4.0,-68.6,-6.0,54.9,24.8,-45.6,-2.0,-24.8,54.9,40.2,23.5,9.0,-22.2,28.0,427.9,21.0,3.0,-19.8,19.0,-66.0,14.2,30.0,-171.0,18.0,-60.0,1.0,68.5,28.0,424.0,233.5,415.6,1350.9,79.8,326.1,522.1,398.1,81.5,24.8,894.5,219.0
AD6,104.0,72.8,228.7,540.8,26.0,264.1,-10.0,56.4,145.9,674.4,122.6,174.0,-85.2,-69.2,582.7,327.1,32.0,157.1,66.8,3.0,35.5,383.1,449.5,31.0,288.0,558.9,340.5,530.0,32.0,23.5,412.5,1220.0,358.5,6320.0,314.9,571.5,193.4,651.8,82.4,517.7,...,20.2,-82.0,14.6,863.5,-41.0,-4.4,-19.9,-10.0,340.5,80.1,40.7,56.4,57.0,-5.8,501.6,22.7,80.1,-0.8,-8.6,-43.0,-27.6,24.3,-138.8,36.7,-60.0,16.8,56.4,61.6,408.8,-8.6,399.3,1036.6,104.0,260.4,294.2,205.2,63.9,160.1,596.8,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
L94,78.1,3.0,132.3,521.8,-3.0,202.2,-5.0,79.0,259.7,533.5,137.0,139.3,-148.7,-49.6,971.6,302.0,29.6,153.3,25.0,5.0,62.7,359.4,1520.7,34.8,206.0,552.3,232.0,441.4,76.4,12.5,471.1,1138.1,937.0,7848.6,695.4,934.8,262.4,769.9,80.0,435.8,...,6.0,-40.1,9.0,133.5,-42.8,-3.0,17.0,-13.7,98.9,34.8,26.0,40.0,24.0,-14.9,395.2,-6.0,41.0,-23.7,20.0,-85.5,13.7,20.0,-252.1,18.0,-73.3,23.0,43.0,55.0,343.0,-98.8,280.6,1362.0,107.1,334.0,812.5,187.9,68.5,317.0,397.2,303.0
L95,68.0,27.2,110.7,425.1,18.5,230.6,-45.3,53.4,202.0,669.4,105.3,153.3,-72.0,14.8,602.3,690.0,25.3,212.2,43.8,29.6,56.3,428.2,2370.9,30.8,290.1,476.6,270.3,484.0,35.0,3.9,774.0,1125.7,944.9,9515.5,471.1,515.4,235.4,961.0,106.2,450.7,...,7.3,-59.8,-4.8,72.0,16.0,-12.4,18.5,-6.5,-2.1,46.2,103.5,68.0,40.2,30.8,386.9,-2.1,40.2,-19.8,9.0,-55.7,-19.8,59.7,-210.0,26.0,-69.6,24.6,25.3,18.5,276.5,-167.6,327.7,428.2,45.0,320.9,295.2,167.0,56.3,318.9,652.1,286.2
L96,35.5,221.5,492.9,433.0,34.0,208.3,-4.1,-187.1,155.2,996.8,179.3,416.3,-43.7,26.8,506.1,1953.0,-20.2,79.8,22.4,78.5,517.1,296.3,1309.0,91.1,363.4,74

In [ ]:
#NORMALISATION

from sklearn.preprocessing import StandardScaler

# create a scaler object
std_scaler = StandardScaler()
std_scaler
# fit and transform the data
df3_unsafe = pd.DataFrame(std_scaler.fit_transform(df_unsafe_trans), columns=df_unsafe_trans.columns)

df3_unsafe

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,...,7091,7092,7093,7094,7095,7096,7097,7098,7099,7100,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129,7130
0,0.173306,-0.229783,-0.310272,0.924642,0.059328,0.953715,-0.501219,-0.784472,-0.627392,-0.173074,-0.060448,-0.583102,0.068880,-0.010238,0.719146,-0.514521,-0.226562,-0.125026,-0.468430,-0.569600,-0.181759,0.392711,-0.449676,-0.176496,-1.376635,0.592264,0.878417,0.024101,-0.354152,-0.372894,-1.022230,-0.361629,0.489421,-0.081966,-0.884895,-1.111420,-0.528265,-1.011536,-0.223823,-0.526717,...,0.479596,0.650018,-0.381971,0.041100,-0.947838,0.150784,0.065410,0.127282,-0.134160,-0.431169,-0.281847,0.083394,-0.687374,0.528651,0.125383,1.069714,-0.508353,-0.264304,0.812929,-0.592859,-0.368208,-0.018684,0.122923,0.729495,-0.268973,-0.425096,-0.853423,-0.292451,0.399837,0.074108,0.406987,0.092035,-0.615932,0.017090,-0.796730,-0.946767,-0.834752,-0.205117,-0.542928,0.985566
1,-0.697890,-0.943698,-0.631097,-0.907199,-0.866055,-0.277880,-1.067216,-0.595944,-0.942539,-0.444162,-0.943169,-0.710922,-0.116318,-1.006679,-0.852327,0.323211,-1.752987,-0.332535,0.117609,-0.462724,-0.389754,-0.255757,-0.599234,-0.367317,0.328043,-0.249272,-0.334962,-0.453261,-0.393836,-0.782648,-0.626916,0.940532,-0.298591,-0.897693,-0.698019,0.272450,0.175236,0.425467,0.148027,0.920872,...,0.222780,-1.015886,0.364546,0.072760,-1.362775,0.377031,0.229525,-0.413734,2.544451,0.939859,-0.201352,2.543419,-0.898903,-0.315556,-0.657819,1.118914,-0.031755,0.697866,-0.427875,-0.168774,-1.166727,-1.243748,0.058737,-0.506930,-1.021339,0.083507,-0.325757,1.375076,0.489326,-0.090972,0.295152,-0.617581,0.551557,-0.300744,1.793828,-0.728734,0.276002,-0.213377,1.989527,-0.867735
2,-0.537552,-0.833758,-0.621216,-0.542400,-1.106486,-0.910382,0.407988,-0.452362,-0.584650,-0.899703,-0.849783,-0.906698,-0.022396,0.453514,-0.627831,-0.671990,0.070434,-0.707375,-0.240909,0.040630,-0.457047,-0.749668,-0.290843,-0.871232,-0.898263,-1.011353,-0.517529,-0.143656,-0.949416,-0.629553,0.946872,0.186429,-0.630786,0.173912,-0.658500,-0.794446,-0.955500,-1.473776,-0.519312,-0.506667,...,-0.380582,0.192170,-0.991998,-0.409576,-0.365897,-1.095127,-0.887852,-0.502472,0.307142,-0.484272,-0.485202,-0.501216,-0.155840,-0.287877,-0.588808,0.271584,0.473439,-0.022418,-0.026332,-0.736346,-1.125246,-0.060638,0.229268,0.002849,0.054762,-0.280241,-0.038473,0.861990,-0.004288,0.243251,-0.022969,-0.476222,-1.826198,-0.428500,0.784389,-0.419168,-0.572354,0.309949,0.267139,0.563228
3,-0.439612,0.152919,-0.535375,-0.124808,-0.147555,-0.372034,0.709050,-0.564730,-0.942539,-0.232950,-0.932052,-0.810428,-0.054144,0.030497,-0.240742,0.754320,1.614126,-0.647817,0.085434,-0.376533,-0.574299,-0.571111,1.516465,-0.519232,-1.158330,0.219767,0.873250,0.438635,0.093713,0.707776,1.719361,0.276690,-1.117674,0.955008,-1.649147,-0.865256,0.054049,-1.599478,-0.939857,0.687627,...,-0.442466,-0.444955,-0.248267,-0.369536,0.264859,-1.494933,-0.688819,-0.926125,-0.087967,0.003312,-0.554399,-0.804347,-1.273146,-0.276805,-0.183832,0.036518,-0.479757,-0.484689,-0.236147,-0.838381,0.212532,0.107178,0.154067,-0.470316,0.317005,-0.537761,0.270309,0.092363,-0.092637,0.627791,0.211131,0.066569,-0.084769,-0.081584,1.427216,2.203400,-0.114636,-0.322707,1.116599,0.237950
4,-0.347990,-0.182468,-0.378205,-0.065356,0.076102,-0.618808,0.004565,-0.614672,-0.978264,-0.550316,-0.194968,-0.364675,-0.113672,-1.401496,-0.732439,-0.745570,0.111876,-0.432745,0.067048,-0.397219,-0.793510,-0.297516,-0.638640,-0.756369,-0.057313,0.534168,0.360857,-0.135236,-0.768002,-0.343626,-0.323895,0.106610,-0.501213,-1.316767,-0.698688,-0.779949,-0.707991,-0.604147,-0.853535,-0.145772,...,-0.068072,-0.814543,0.325549,7.159734,-0.580432,-0.218031,-1.313852,-0.502472,0.741763,0.966410,-0.311503,-0.014874,0.874685,-0.744579,0.210444,0.082985,1.357529,-0.144257

In [ ]:
for i in range(0,len(df3_unsafe)) :
  for j in range(0,len(df3_unsafe.columns)):
    if -1.0 < df3_unsafe.iloc[i,j] > 0.0:
      df3_unsafe.iloc[i, j] = gn[j]
    else:
      df3_unsafe.iloc[i, j] = 0
print(df3_unsafe)

      2    3       4       5    6     ...   7126      7127     7128   7129     7130
0   GABRA3    0       0  SEMA3C  GML  ...      0         0        0      0  RAB9P40
1        0    0       0       0    0  ...      0  Gene7125        0  UREB1        0
2        0    0       0       0    0  ...      0         0  SLC6A14  UREB1  RAB9P40
3        0  OMD       0       0    0  ...  ATP7A         0        0  UREB1  RAB9P40
4        0    0       0       0  GML  ...  ATP7A         0  SLC6A14      0        0
..     ...  ...     ...     ...  ...  ...    ...       ...      ...    ...      ...
81       0    0       0       0    0  ...  ATP7A         0  SLC6A14      0  RAB9P40
82       0    0       0       0    0  ...      0         0  SLC6A14  UREB1  RAB9P40
83       0  OMD  GS3686       0  GML  ...      0         0        0      0        0
84       0    0       0       0    0  ...      0         0        0      0        0
85       0    0       0       0  GML  ...  ATP7A         0        0      0  

In [ ]:
df3_unsafe = df3_unsafe.replace(0, np.nan)
df3_unsafe

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,...,7091,7092,7093,7094,7095,7096,7097,7098,7099,7100,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129,7130
0,GABRA3,NaN,NaN,SEMA3C,GML,MKNK1,NaN,NaN,NaN,NaN,NaN,NaN,Gene12,NaN,RNF4,NaN,NaN,NaN,NaN,NaN,NaN,GMFB,NaN,NaN,NaN,KIAA0320,CLOCK,MADD,NaN,NaN,NaN,NaN,SCYA21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,GFPT1,FLJ11618,NaN,PTPRR,NaN,H2BFQ,Gene7095,H1FT,NaN,NaN,NaN,H4FJ,NaN,SERPINI1,H2BFD,H3FK,NaN,NaN,PPP1R11,NaN,NaN,NaN,DNAH9,DNAH9,NaN,NaN,NaN,NaN,BRD2,GSTT1,HBZ,HBA2,NaN,SSTR3,NaN,NaN,NaN,NaN,NaN,RAB9P40
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PLAB,NaN,NaN,PCDHGB4,NaN,NaN,NaN,NaN,NaN,KIAA0317,NaN,NaN,NaN,NaN,NaN,NaN,CTNND1,NaN,NaN,NaN,PSMD11,PSMD12,PSMD9,CDC7L1,TLK2,...,GFPT1,NaN,Gene7091,PTPRR,NaN,H2BFQ,Gene7095,NaN,H2BFH,Gene7098,NaN,H4FJ,NaN,NaN,NaN,H3FK,NaN,H2AFC,NaN,NaN,NaN,NaN,DNAH9,NaN,NaN,DNAH9,NaN,KL,BRD2,NaN,HBZ,NaN,Gene7121,NaN,E46L,NaN,Gene7125,NaN,UREB1,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,OGG1,NaN,NaN,NaN,NaN,NaN,NaN,RES4-25,NaN,NaN,ARNTL,NaN,NaN,PCDHGA12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARHGEF12,CTNND1,NaN,RPLP2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,FLJ11618,NaN,NaN,NaN,NaN,NaN,NaN,H2BFH,NaN,NaN,NaN,NaN,NaN,NaN,H3FK,H2AFM,NaN,NaN,NaN,NaN,NaN,DNAH9,DNAH9,DNAH9,NaN,NaN,KL,NaN,GSTT1,NaN,NaN,NaN,NaN,E46L,NaN,NaN,SLC6A14,UREB1,RAB9P40
3,NaN,OMD,NaN,NaN,NaN,NaN,OGG1,NaN,NaN,NaN,NaN,NaN,NaN,RES4-25,NaN,PLAB,ARNTL,NaN,PCDHGB4,NaN,NaN,NaN,AQP3,NaN,NaN,KIAA0320,CLOCK,MADD,KIAA0367,KIAA0368,ARHGEF12,CTNND1,NaN,RPLP2,NaN,NaN,PSMD12,NaN,NaN,TLK2,...,NaN,NaN,NaN,NaN,ACADS,NaN,NaN,NaN,NaN,Gene7098,NaN,NaN,NaN,NaN,NaN,H3FK,NaN,NaN,NaN,NaN,CKS1,DNAH7,DNAH9,NaN,DNAH9,NaN,CYB5-M,KL,NaN,GSTT1,HBZ,HBA2,NaN,NaN,E46L,ATP7A,NaN,NaN,UREB1,RAB9P40
4,NaN,NaN,NaN,NaN,GML,NaN,OGG1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARNTL,NaN,PCDHGB4,NaN,NaN,NaN,NaN,NaN,NaN,KIAA0320,CLOCK,NaN,NaN,NaN,NaN,CTNND1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Gene7091,PTPRR,NaN,NaN,NaN,NaN,H2BFH,Gene7098,NaN,NaN,H4FL,NaN,H2BFD,H3FK,H2AFM,NaN,NaN,NaN,NaN,NaN,DNAH9,DNAH9,DNAH9,NaN,CYB5-M,KL,NaN,GSTT1,HBZ,NaN,Gene7121,NaN,NaN,ATP7A,NaN,SLC6A14,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,NaN,NaN,NaN,NaN,NaN,NaN,OGG1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RNF4,NaN,ARNTL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KIAA0320,NaN,NaN,NaN,NaN,NaN,NaN,SCYA21,NaN,STXBP2,PSMD11,PSMD12,PSMD9,NaN,NaN,...,NaN,FLJ11618,Gene7091,PTPRR,NaN,NaN,NaN,NaN,H2BFH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H2AFM,NaN,NaN,NaN,CKS1,NaN,NaN,NaN,DNAH9,DNAH9,NaN,KL,NaN,NaN,NaN,HBA2,Gene7121,SSTR3,E46L,ATP7A,NaN,SLC6A14,NaN,RAB9P40
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gene12,NaN,NaN,NaN,NaN,NaN,NaN,PCDHGA12,NaN,NaN,AQP3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARHGEF12,NaN,SCYA21,RPLP2,STXBP2,NaN,NaN,PSMD9,NaN,NaN,...,NaN,NaN,NaN,NaN,ACADS,NaN,Gene7095,NaN,NaN,Gene7098,TPM1,H4FJ,H4FL,NaN,NaN,NaN,H2AFM,NaN,NaN,NaN,NaN,DNAH7,DNAH9,NaN,DNAH9,DNAH9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SLC6A14,UREB1,RAB9P40
83,NaN,OMD,GS3686,NaN,GML,NaN,OGG1,NaN,NaN,RES4-22,SH3BP2,Gene11,Gene12,RES4-25,NaN,PLAB,NaN,NaN,NaN,PCDHGA12,H4FM,NaN,NaN,KIAA0316,KIAA0317,KIAA0320,CLOCK,MADD,KIAA0367,KIAA0368,NaN,NaN,SCYA21,NaN,STXBP2,NaN,NaN,NaN,NaN,NaN,...,NaN,FLJ11618,NaN,NaN,NaN,H2BFQ,NaN,H1FT,NaN,NaN,TPM1,NaN,NaN,NaN,H2BFD,NaN,H2AFM,NaN,PPP1R11,DNAH9,CKS1,NaN,DNAH9,DNAH9,DNAH9,DNAH9,CYB5-M,NaN,BRD2,NaN,NaN,NaN,NaN,SSTR3,NaN,NaN,NaN,NaN,NaN,NaN
84,NaN,NaN,NaN,NaN,NaN,MKNK1,OGG1,NaN,NaN,NaN,NaN,NaN,NaN,RES4-25,NaN,NaN,ARNTL,NaN,NaN,NaN,NaN,NaN,AQP3,NaN,NaN,KIAA0320,CLOCK,NaN,KIAA0367,KIAA0368,ARHGEF12,CTNND1,SCYA21,RPLP2,STXBP2,NaN,PSMD12,PSMD9,NaN,N

In [ ]:
records_unsafe = []
for i in range(0, len(df3_unsafe)):
    records_unsafe.append([str(df3_unsafe.values[i,j]) for j in range(0, len(df3_unsafe.columns))])

In [ ]:
association_rules_unsafe = apriori(records_unsafe, min_support=0.005, min_confidence=0.3, min_lift=3, min_length=2,max_length = 2)
association_results_unsafe = list(association_rules_unsafe)


In [ ]:
support_unsafe = []
conf_unsafe = []

for item in association_results_unsafe:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))
    support_unsafe.append(float(item[1]))
    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    conf_unsafe.append(float(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

Rule: AKR1C3 -> ABCC2
Support: 0.13953488372093023
Confidence: 0.75
Lift: 3.0714285714285716
Rule: CPS1 -> ABCC2
Support: 0.09302325581395349
Confidence: 0.5
Lift: 3.071428571428571
Rule: FGB -> ABCC2
Support: 0.06976744186046512
Confidence: 0.375
Lift: 3.225
Rule: Gene546 -> ABCC2
Support: 0.10465116279069768
Confidence: 0.5625
Lift: 3.0234375
Rule: TF -> ABCC2
Support: 0.046511627906976744
Confidence: 0.6666666666666666
Lift: 3.583333333333333
Rule: FHL1 -> ADH2
Support: 0.22093023255813954
Confidence: 0.8636363636363636
Lift: 3.0946969696969697
Rule: APOA2 -> AF1Q
Support: 0.046511627906976744
Confidence: 0.8
Lift: 4.0470588235294125
Rule: AF1Q -> NTS
Support: 0.046511627906976744
Confidence: 0.6666666666666666
Lift: 3.372549019607843
Rule: AF1Q -> PCSK1
Support: 0.05813953488372093
Confidence: 0.625
Lift: 3.1617647058823533
Rule: AF1Q -> S100A7
Support: 0.03488372093023256
Confidence: 0.6
Lift: 3.0352941176470587
Rule: SNAP25 -> AGTR1
Support: 0.13953488372093023
Confidence: 0.6666

In [ ]:
with open("un-healthy.txt", "w") as txt_file:
    for item in association_results_unsafe:
      pair = item[0] 
      items = [x for x in pair]
      txt_file.write("Rule: " + items[0] + " -> " + items[1] + "\t"+"Support: " + str(item[1])+"\t"+"Confidence: " + str(item[2][0][2])+  "\n")
   

##**Common in Both**#

In [ ]:
 healthy_val = []
 unhealthy_val = []
 for item in association_results:
      pair = item[0] 
      items = [x for x in pair]
      healthy_val.append(str(items[0]) + " -> " + str(items[1]))

for item in association_results_unsafe:
  pair = item[0] 
  items = [x for x in pair]
  unhealthy_val.append(str(items[0]) + " -> " + str(items[1]))

matched = []
for i in healthy_val:
  if i in unhealthy_val:
    matched.append(i)

print(*matched, sep = '\n')



KRT17 -> S100A7
LTF -> PI3
LTF -> S100A7
PI3 -> S100A7
SERPINA3 -> S100A7


##**Only in Normal State**##

In [ ]:
not_in_unhealthy = []
for i in healthy_val:
  if i not in unhealthy_val:
    not_in_unhealthy.append(i)

print(*not_in_unhealthy, sep = '\n')

Streaming output truncated to the last 5000 lines.
CD2 -> GZMA
CD2 -> GZMK
CD2 -> Gene1394
CD2 -> Gene149
CD2 -> Gene151
CD2 -> Gene152
CD2 -> Gene2979
CD2 -> IFI16
IFRD2 -> CD2
CD2 -> IL2RG
INDO -> CD2
CD2 -> ITGAL
LGALS2 -> CD2
LMNB1 -> CD2
CD2 -> LOC58504
CD2 -> MIG
MS4A2 -> CD2
NK4 -> CD2
CD2 -> PSMB10
CD2 -> PSMB8
CD2 -> PSMB9
CD2 -> PSME1
CD2 -> PSME2
PTGDR -> CD2
PTPRCAP -> CD2
SCYA19 -> CD2
CD2 -> SCYB10
CD2 -> SCYB11
CD2 -> SEMA4D
CD2 -> STAF50
STAT1 -> CD2
CD2 -> TNFRSF5
CD2 -> TRA@
CD2 -> TXN
CD2 -> WARS
CD24 -> CDH15
CD24 -> CYP
CD24 -> FKBP2
CD24 -> GPRK7
CD24 -> Gene2979
CD24 -> Gene5908
CD24 -> HDLBP
CD24 -> ICAM3
CD24 -> KIAA0125
CD24 -> KRT13
CD24 -> LGALS2
CD24 -> MPI
MS4A2 -> CD24
CD24 -> OPRD1
CD24 -> PAFAH1B3
CD24 -> PIM2
CD24 -> POU2AF1
RFX2 -> CD24
CD24 -> SLU7
SSR4 -> CD24
CD24 -> TNFRSF7
CD24 -> XBP1
CD36L1 -> DEFB2
E4F1 -> CD36L1
CD36L1 -> LCN2
CD36L1 -> LOC54453
LTF -> CD36L1
MMP1 -> CD36L1
CD36L1 -> MMP12
CD36L1 -> PI3
CD36L1 -> S100A2
CD36L1 -> S100A7
SERPI

##**Only in Carcinogenic State**##

In [ ]:
not_in_healthy = []
for i in unhealthy_val:
  if i not in healthy_val:
    not_in_healthy.append(i)

print(*not_in_healthy, sep = '\n')

AKR1C3 -> ABCC2
CPS1 -> ABCC2
FGB -> ABCC2
Gene546 -> ABCC2
TF -> ABCC2
FHL1 -> ADH2
APOA2 -> AF1Q
AF1Q -> NTS
AF1Q -> PCSK1
AF1Q -> S100A7
SNAP25 -> AGTR1
TF -> AGTR1
AKR1C4 -> AHSG
APOA1 -> AHSG
APOA2 -> AHSG
APOH -> AHSG
ASGR1 -> AHSG
AHSG -> CCK
CHGB -> AHSG
CPB2 -> AHSG
CTAG1 -> AHSG
AHSG -> DUSP9
ENPP1 -> AHSG
AHSG -> F5
FOXD1 -> AHSG
GC -> AHSG
HPD -> AHSG
IAPP -> AHSG
IGFBP1 -> AHSG
INSM1 -> AHSG
AHSG -> ITIH2
KRT14 -> AHSG
NEUROD1 -> AHSG
PDE6C -> AHSG
REG1A -> AHSG
AHSG -> SERPIND1
AHSG -> SNAP25
TAC1 -> AHSG
AHSG -> TF
TTR -> AHSG
VTN -> AHSG
AKAP12 -> NTS
AKR1C3 -> AKR1C1
AKR1C1 -> DIA4
Gene546 -> AKR1C1
AKR1C3 -> CBR1
FGB -> AKR1C3
GCLM -> AKR1C3
GPX2 -> AKR1C3
Gene546 -> AKR1C3
AKR1C3 -> TXNRD1
AKR1C3 -> UGT1A6
APOA2 -> AKR1C4
AKR1C4 -> APOB
AKR1C4 -> APOH
AKR1C4 -> ATP12A
CPB2 -> AKR1C4
AKR1C4 -> GC
AKR1C4 -> HPD
AKR1C4 -> NTS
PGC -> AKR1C4
PLA2G1B -> AKR1C4
AKR1C4 -> SERPIND1
AKR1C4 -> TF
AKR1C4 -> VTN
TFF2 -> ALDH3
SST -> ALDH8
ANK1 -> ATP12A
ATP12A -> ANXA8
ANXA8 -> K

##**Degree of Association**##

In [ ]:
from statistics import mean
support_mean_safe = mean(support_safe)
support_mean_unsafe = mean(support_unsafe)
conf_mean_safe = mean(conf_safe)
conf_mean_unsafe = mean(conf_unsafe)
strong_safe = []
strong_unsafe = []
for i in range(len(healthy_val)):
  if(support_safe[i] > support_mean_safe and conf_safe[i] < conf_mean_safe):
    strong_safe.append(healthy_val[i])

for i in range(len(unhealthy_val)):
  if(support_unsafe[i] > support_mean_unsafe and conf_unsafe[i] <conf_mean_unsafe):
    strong_unsafe.append(unhealthy_val[i])


##**Output in a File**##

In [ ]:
#matched
with open("Matched.txt", "w") as txt_file:
    for item in matched:
      txt_file.write(item + "\n")

#Not in UnHealthy
with open("Not_in_UnHealthy.txt", "w") as txt_file:
    for item in not_in_unhealthy:
      txt_file.write(item + "\n")

#Not in Healthy
with open("Not_in_Healthy.txt", "w") as txt_file:
    for item in not_in_healthy:
      txt_file.write(item + "\n")

#Strongly associated in only Healthy state 
with open("Strongly associated in only Healthy state .txt", "w") as txt_file:
    for item in strong_safe:
      txt_file.write(item + "\n")

#Strongly associated in only UnHealthy state 
with open("Strongly associated in only UnHealthy state .txt", "w") as txt_file:
    for item in strong_unsafe:
      txt_file.write(item + "\n")

